Download GNPS data <br>
Replace Job ID below with your GNPS job ID:

In [1]:
! curl -d "" "https://gnps.ucsd.edu/ProteoSAFe/DownloadResult?task=b817262cb6114e7295fee4f73b22a3ad&view=download_cytoscape_data" -o GNPS_output_graphML_FeatureBased.zip

from zipfile import ZipFile
zf = ZipFile("GNPS_output_graphML_FeatureBased.zip", 'r')
zf.extractall("GNPS_output_graphML_FeatureBased")
zf.close()

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 2237k    0 2237k    0     0  2237k      0 --:--:-- --:--:-- --:--:-- 4336k
100 3607k    0 3607k    0     0  3607k      0 --:--:-- --:--:-- --:--:-- 4442k


specify your NAP job ID:

In [1]:
nap_id = 'c4bb6b8be9e14bdebe87c6ef3abe11f6'

load libraries

In [2]:
from pyMolNetEnhancer import *
import pandas as pd
import os
import csv 
import json
from networkx import *
from pathlib import Path  #path to MarvinSuite's molconvert used to convert SMILES to InchiKeys
pathMolCov = Path('C:/Program Files/ChemAxon/MarvinSuite/bin')
pathJAVA = Path('C:/Program Files/Java/jre1.8.0_211')
os.environ['PATH'] = str(pathMolCov)
os.environ['JAVA_HOME'] = str(pathJAVA)

In [3]:
if 'clusterinfo_summary' in os.listdir('GNPS_output_graphML_FeatureBased/') and 'DB_result' in os.listdir('GNPS_output_graphML_FeatureBased/'):
    netfile = 'GNPS_output_graphML_FeatureBased/clusterinfo_summary/' + str(os.listdir('GNPS_output_graphML_FeatureBased/clusterinfo_summary/')[0]) 
    gnpslibfile = 'GNPS_output_graphML_FeatureBased/DB_result/'+ str(os.listdir('GNPS_output_graphML_FeatureBased/DB_result/')[0]) 

elif 'clusterinfosummarygroup_attributes_withIDs_withcomponentID' in os.listdir('GNPS_output_graphML_FeatureBased/'):
    netfile = 'GNPS_output_graphML_FeatureBased/clusterinfosummarygroup_attributes_withIDs_withcomponentID/' + str(os.listdir('GNPS_output_graphML_FeatureBased/clusterinfosummarygroup_attributes_withIDs_withcomponentID/')[0])
    gnpslibfile = 'GNPS_output_graphML_FeatureBased/result_specnets_DB/'+ str(os.listdir('GNPS_output_graphML_FeatureBased/result_specnets_DB/')[0])

else:
    netfile = 'GNPS_output_graphML_FeatureBased/clusterinfosummary/' + str(os.listdir('GNPS_output_graphML_FeatureBased/clusterinfosummary/')[0])
    gnpslibfile = 'GNPS_output_graphML_FeatureBased/result_specnets_DB/'+ str(os.listdir('GNPS_output_graphML_FeatureBased/result_specnets_DB/')[0]) 

In [4]:
nap = pd.read_csv("http://proteomics2.ucsd.edu/ProteoSAFe/DownloadResultFile?task="+nap_id+"&block=main&file=final_out/node_attributes_table.tsv", sep = "\t")
gnpslib = pd.read_csv(gnpslibfile, sep='\t')

add all chemical structural information output as dataframe items in list

In [5]:
matches = [gnpslib, nap]

In [6]:
out = unique_smiles(matches)

In [7]:
out['df'].to_csv("SMILES.csv", quoting=csv.QUOTE_NONE, escapechar='&')

convert SMILES to InchiKeys

In [8]:
! molconvert inchikey:SAbs SMILES.csv{csv:strucSMILES} -o InchiKeys_FeatureBased.txt

SMILES.csv: error in molecule 1460: Cannot recognize format by any of the supported molecular file format recognizers.
Unrecognized file contents:
C/C=c/1(c2=CC=Nc2c1Cc1cc(cc(c1)C)C)C[C@H]1[C@H]([C@@H]2[C@@]3(CCC(=O)[C@@]([C@@H]3CC[C@@]2([C@@]2(C1=C(C(=O)C2)[C@H](C)C[C@H]([C@@H]1C(O1)(C)C)O)C)C)(C)C1CCCC1)C)O...



In [9]:
smiles_df = pd.read_csv('SMILES.csv')
len(smiles_df)

6248

In [10]:
len(pd.read_csv('InchiKeys_FeatureBased.txt', sep='\t', header=None))

1459

In [11]:
ikeys = pd.read_csv("InchiKeys_FeatureBased.txt",  sep='\t',header = None)

In [12]:
len(ikeys)

1459

In [13]:
out['df']['inchikey'] = ikeys
inchi_dic = make_inchidic(out)

In [14]:
len(out['df']['inchikey'])

6248

In [15]:
len(inchi_dic)

1705

In [16]:
ikeys.columns = ['InChIKey']
ikeys.to_csv('InchiKeys_FeatureBased.txt', quoting=csv.QUOTE_NONE, escapechar='&')

retrieve ClassyFire classifcations

In [19]:
get_classifications("InchiKeys_FeatureBased.txt")

1000


exception calling callback for <Future at 0x2068f170080 state=finished raised TerminatedWorkerError>
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\joblib\externals\loky\_base.py", line 625, in _invoke_callbacks
    callback(self)
  File "C:\ProgramData\Anaconda3\lib\site-packages\joblib\parallel.py", line 309, in __call__
    self.parallel.dispatch_next()
  File "C:\ProgramData\Anaconda3\lib\site-packages\joblib\parallel.py", line 731, in dispatch_next
    if not self.dispatch_one_batch(self._original_iterator):
  File "C:\ProgramData\Anaconda3\lib\site-packages\joblib\parallel.py", line 759, in dispatch_one_batch
    self._dispatch(tasks)
  File "C:\ProgramData\Anaconda3\lib\site-packages\joblib\parallel.py", line 716, in _dispatch
    job = self._backend.apply_async(batch, callback=cb)
  File "C:\ProgramData\Anaconda3\lib\site-packages\joblib\_parallel_backends.py", line 510, in apply_async
    future = self._workers.submit(SafeFunction(func))


TerminatedWorkerError: A worker process managed by the executor was unexpectedly terminated. This could be caused by a segmentation fault while calling the function or by an excessive memory usage causing the Operating System to kill the worker.

In [21]:
with open("all_json.json") as tweetfile:
    jsondic = json.loads(tweetfile.read())


df = make_classy_table(jsondic)
df = df.rename(columns = {'class':'CF_class','smiles':'SMILES'})
net = pd.read_csv(netfile,  sep='\t')


final = molfam_classes(net,df,inchi_dic)

In [22]:
final.head()

,cluster index,CF_componentindex,CF_NrNodes,CF_kingdom,CF_kingdom_score,CF_superclass,CF_superclass_score,CF_class,CF_class_score,CF_subclass,CF_subclass_score,CF_Dparent,CF_Dparent_score,CF_MFramework,CF_MFramework_score
0,1,343,4,no matches,,no matches,,no matches,,no matches,,no matches,,no matches,
1,3,343,4,no matches,,no matches,,no matches,,no matches,,no matches,,no matches,
2,20,343,4,no matches,,no matches,,no matches,,no matches,,no matches,,no matches,
3,1959,343,4,no matches,,no matches,,no matches,,no matches,,no matches,,no matches,
4,2,S1,1,no matches,,no matches,,no matches,,no matches,,no matches,,no matches,


write ClassyFire results per molecular family to file. To visualize chemical classes within the mass spectral molecular network, this file can be imported as table into Cytoscape.

In [13]:
final.to_csv("ClassyFireResults_Network_FeatureBased.txt", sep = '\t', index = False)

how many InChIKeys could not be classified? (percentage of total submitted InChIKeys)

In [14]:
len(set(list(ikeys.InChIKey)) - set(list(df.inchikey)))/len(set(list(ikeys.InChIKey)))

0.652014652014652

which InChIKeys could not be classified?

In [15]:
set(list(ikeys.InChIKey)) - set(list(df.inchikey))

{'InChIKey=AAUZLHJDJNJJDM-YYNGXESYSA-N',
 'InChIKey=ABMLTSFSLUQUFY-XUAJAPSHSA-N',
 'InChIKey=AFGUVBVUFZMJMX-UHFFFAOYSA-N',
 'InChIKey=APKQIGMSWOACOS-MSUUIHNZSA-N',
 'InChIKey=AURCBSWGPREGRQ-AQNFZCBMSA-M',
 'InChIKey=AUWWRCVWPCJXBI-GJZGRUSLSA-O',
 'InChIKey=AYQMHVMNRNJETE-FIXSFTCYSA-N',
 'InChIKey=BCHQLIFQFCPGHF-QGZVFWFLSA-M',
 'InChIKey=BEKLYIAFACFFDO-UPBMBJRSSA-N',
 'InChIKey=BEVFYPNHCDGTJQ-UHFFFAOYSA-N',
 'InChIKey=BFQWBQIYMSVPOL-HEUYONADSA-N',
 'InChIKey=BLSNFMIGHPIPFW-ZUQMNNKGSA-N',
 'InChIKey=BQGLBJMHEQGMIA-UHFFFAOYSA-N',
 'InChIKey=BQHCMGWMRJLGIG-QSQNZYKQSA-P',
 'InChIKey=BROOAJNAIWOCEO-JGCGQSQUSA-N',
 'InChIKey=BURLCNUJBHPOOO-RUWUMMDESA-N',
 'InChIKey=BXFOFFBJRFZBQZ-UHFFFAOYSA-N',
 'InChIKey=CCOVPWPRULOZOO-PLACYDAFSA-N',
 'InChIKey=CGEPVABIQUNBOM-TYNPDJOYSA-N',
 'InChIKey=CHHZYSAKMRQNTM-FUHWJXTLSA-O',
 'InChIKey=CIOUQKGNHMSZRR-UKZDDFKXSA-N',
 'InChIKey=CJQYOTZFGCMSNR-VCZQVZGSSA-N',
 'InChIKey=CRXKETATQYMLCU-BQAXFXPJSA-Q',
 'InChIKey=CSIFRWRBOOQRBP-UZJFSXEZSA-N',
 'InChIKey=DESNV

create graphML file

In [18]:
if any("FEATURE" in s for s in os.listdir('GNPS_output_graphML_FeatureBased/')):
    graphMLfile = 'GNPS_output_graphML_FeatureBased/' + [x for x in os.listdir('GNPS_output_graphML_FeatureBased/') if 'FEATURE' in x][0]
    graphML = read_graphml(graphMLfile)
    graphML_classy = make_classyfire_graphml(graphML,final)
    nx.write_graphml(graphML_classy, "ClassyFireResults_Network_FeatureBased.graphml", infer_numeric_types = True)
elif any("METABOLOMICS" in s for s in os.listdir('GNPS_output_graphML_FeatureBased/')):
    graphMLfile = 'GNPS_output_graphML_FeatureBased/' + [x for x in os.listdir('GNPS_output_graphML_FeatureBased/') if 'METABOLOMICS' in x][0]
    graphML = read_graphml(graphMLfile)
    graphML_classy = make_classyfire_graphml(graphML,final)
    nx.write_graphml(graphML_classy, "ClassyFireResults_Network_FeatureBased.graphml", infer_numeric_types = True)
else:
    print('There is no graphML file for this GNPS molecular network job')